# Read The DataSet

In [1]:
A = open("D:/GitHub_Projects/Sentence_Completion_Using_LSTM/sherlock.txt",encoding="utf8")
data = A.read()

In [2]:
#data

# Tokenize Data and PreProcessing of Data

In [3]:
Q = []
import re
for i in data.split(" "):
    w = re.sub("[^a-zA-Z]","",i)
    w = w.upper()
    if(w!=""):
        Q.append(w)

As the text file contains large amount of data due to which memory error will occur at the time of Neural Network.So I have taken only sample amount of Data to get the results.

In [4]:
Q = Q[0:1000]

# Find the Unique Words

In [5]:
from numpy import unique
unique_words = unique(Q)

# Create X and Y in string format

In [6]:
len(Q)

1000

In [7]:
Xstr = []
Ystr = []
for i in range(4,len(Q),1):
    w1 = Q[i-4]
    w2 = Q[i-3]
    w3 = Q[i-2]
    w4 = Q[i-1]
    wy = Q[i]
    line = w1 + " " + w2 + " " + w3 + " " + w4
    Xstr.append(line)
    Ystr.append(wy)

In [8]:
Q[0:20]

['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN',
 'DOYLETHIS',
 'EBOOK',
 'IS',
 'FOR',
 'THE',
 'USE',
 'OF',
 'ANYONE',
 'ANYWHERE',
 'AT']

In [9]:
from pandas import DataFrame
DataFrame([Xstr,Ystr]).T

,0,1
0,PROJECT GUTENBERGS THE ADVENTURES,OF
1,GUTENBERGS THE ADVENTURES OF,SHERLOCK
2,THE ADVENTURES OF SHERLOCK,HOLMES
3,ADVENTURES OF SHERLOCK HOLMES,BY
4,OF SHERLOCK HOLMES BY,ARTHUR
...,...,...
991,BY SIX ALMOST PARALLEL,CUTS
992,SIX ALMOST PARALLEL CUTS,OBVIOUSLY
993,ALMOST PARALLEL CUTS OBVIOUSLY,THEY
994,PARALLEL CUTS OBVIOUSLY THEY,HAVE


# Declare Arrays for X and Y

In [10]:
lines = len(Xstr)
n_uniques = len(unique_words)
seq = 4

In [11]:
lines

996

In [12]:
n_uniques

515

In [13]:
seq

4

In [14]:
from numpy import zeros
X_arr = zeros((lines,n_uniques,seq),dtype="bool") 
Y_arr = zeros((lines,n_uniques))

# Data Preparation for X_arr and Y_arr

In [15]:
line_enum = list(enumerate(Xstr))

In [16]:
pos_finder = {}
for i,j in list(enumerate(unique_words)):
    pos_finder[j] = i

In [17]:
pos_finder["SHERLOCK"]

395

In [18]:
for line_no,line in line_enum:
    for seq,word in enumerate(line.split(" ")):
        X_arr[line_no,pos_finder[word],seq] = 1

In [19]:
Y_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
for i,j in enumerate(Ystr):
    pos = pos_finder[j]
    Y_arr[i,pos]=1

In [21]:
Y_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
X_arr.shape

(996, 515, 4)

# Create a NN

In [23]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.activations import softmax
from keras.layers.core import Activation

In [24]:
nn = Sequential()
nn.add(LSTM(200,input_shape=(515,4)))
nn.add(Dense(515))
nn.add(Activation("softmax"))

In [25]:
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model = nn.fit(X_arr,Y_arr,epochs=2,validation_split=0.01)

Epoch 1/2
31/31 [==============================] - 18s 531ms/step - loss: 6.0714 - accuracy: 0.0456 - val_loss: 6.8794 - val_accuracy: 0.0000e+00
Epoch 2/2
31/31 [==============================] - 17s 557ms/step - loss: 5.7400 - accuracy: 0.0578 - val_loss: 7.4285 - val_accuracy: 0.0000e+00


In [33]:
q = input("Enter sentence:(4 words)")
q = q.upper()

Enter sentence:(4 words)adventures of sherlock holmes


Enter the phrase which completion we need.  

In [34]:
q

'ADVENTURES OF SHERLOCK HOLMES'

In [35]:
inp = zeros((1,515,4),dtype="bool")

for word_no,word in enumerate(q.split(" ")):
    inp[0,pos_finder[word],word_no]=1

In [36]:
import pandas as pd
B = pd.DataFrame(zip(unique_words,list(nn.predict(inp)[0])),columns=["Word","Prob"])
x = B.Prob.max()

1/1 [==============================] - 0s 31ms/step


In [38]:
B[B.Prob == x].Word.values[0]

'THE'